<a href="https://colab.research.google.com/github/MIT-LCP/2019_tokyo_datathon/blob/master/04_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# eICU Collaborative Research Database

# Notebook 4: Prediction

This notebook explores how a decision trees can be trained to predict in-hospital mortality of patients.


## Load libraries and connect to the database

In [0]:
# Import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# model building
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import metrics
from sklearn import impute

from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn.model_selection import cross_val_score

# Make pandas dataframes prettier
from IPython.display import display, HTML, Image
plt.rcParams.update({'font.size': 20})
%matplotlib inline
plt.style.use('ggplot')

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [0]:
# authenticate
auth.authenticate_user()

In [0]:
# Set up environment variables
project_id='datathonjapan2019'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

To make our lives easier, we'll also install and import a set of helper functions from the `datathon2` package. We will be using the following functions from the package:
- `plot_model_pred_2d`: to visualize our data, helping to display a class split assigned by a tree vs the true class.
- `run_query()`: to run an SQL query against our BigQuery database and assign the results to a dataframe. 


In [0]:
!pip install datathon2

In [0]:
import datathon2 as dtn
import pydotplus
from tableone import TableOne

In this notebook we'll be looking at tree models, so we'll now install a package for visualizing these models.

In [0]:
!apt-get install graphviz -y

## Load the patient cohort

Let's extract a cohort of patients admitted to the ICU from the emergency department.  We link demographics data from the `patient` table to severity of illness score data in the `apachepatientresult` table. We exclude readmissions and neurological patients to help create a population suitable for our demonstration.

In [0]:
# Link the patient, apachepatientresult, and apacheapsvar tables on patientunitstayid
# using an inner join.
query = """
SELECT p.unitadmitsource, p.gender, p.age, p.unittype, p.unitstaytype, 
    a.actualhospitalmortality, a.acutePhysiologyScore, a.apacheScore
FROM `physionet-data.eicu_crd_demo.patient` p
INNER JOIN `physionet-data.eicu_crd_demo.apachepatientresult` a
ON p.patientunitstayid = a.patientunitstayid
WHERE a.apacheversion LIKE 'IVa'
AND LOWER(p.unitadmitsource) LIKE "%emergency%"
AND LOWER(p.unitstaytype) LIKE "admit%"
AND LOWER(p.unittype) NOT LIKE "%neuro%";
"""

cohort = dtn.run_query(query,project_id)

In [0]:
cohort.head()

## Prepare the data for analysis

Before continuing, we want to review our data, paying attention to factors such as:
- data types (for example, are values recorded as characters or numerical values?) 
- missing data
- distribution of values

In [0]:
# dataset info
print(cohort.info())

In [0]:
# Encode the categorical data
encoder = preprocessing.LabelEncoder()
cohort['gender_code'] = encoder.fit_transform(cohort['gender'])
cohort['actualhospitalmortality_code'] = encoder.fit_transform(cohort['actualhospitalmortality'])


In [0]:
# Handle the deidentified ages
cohort['age'] = pd.to_numeric(cohort['age'], downcast='integer', errors='coerce')
cohort['age'] = cohort['age'].fillna(value=91.5)

In [0]:
# Preview the encoded data
cohort[['gender','gender_code']].head()

In [0]:
# Check the outcome variable
cohort['actualhospitalmortality_code'].unique()

In [0]:
# View summary statistics
pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
TableOne(cohort,groupby='actualhospitalmortality')

From these summary statistics, we can see that the average age is higher in the group of patients who do not survive. What other differences do you see?

## Create our train and test sets

We only focus on two variables for our analysis, age and acute physiology score. Limiting ourselves to two variables will make it easier to visualize our models.

In [0]:
features = ['age','acutePhysiologyScore']
outcome = 'actualhospitalmortality_code'

X = cohort[features]
y = cohort[outcome]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [0]:
# Review the number of cases in each set
print("Train data: {}".format(len(X_train)))
print("Test data: {}".format(len(X_test)))

## Decision trees

Let's build the simplest tree model we can think of: a classification tree with only one split. Decision trees of this form are commonly referred to under the umbrella term Classification and Regression Trees (CART) [1]. 

While we will only be looking at classification here, regression isn't too different. After grouping the data (which is essentially what a decision tree does), classification involves assigning all members of the group to the majority class of that group during training. Regression is the same, except you would assign the average value, not the majority. 

In the case of a decision tree with one split, often called a "stump", the model will partition the data into two groups, and assign classes for those two groups based on majority vote. There are many parameters available for the DecisionTreeClassifier class; by specifying max_depth=1 we will build a decision tree with only one split - i.e. of depth 1.

[1] L. Breiman, J. Friedman, R. Olshen, and C. Stone. Classification and Regression Trees. Wadsworth, Belmont, CA, 1984.

In [0]:
# specify max_depth=1 so we train a stump, i.e. a tree with only 1 split
mdl = tree.DecisionTreeClassifier(max_depth=1)

# fit the model to the data - trying to predict y from X
mdl = mdl.fit(X_train,y_train)

Our model is so simple that we can look at the full decision tree.

In [0]:
graph = dtn.create_graph(mdl,feature_names=features)
Image(graph.create_png())

Here we see three nodes: a node at the top, a node in the lower left, and a node in the lower right.

The top node is the root of the tree: it contains all the data. Let's read this node bottom to top:
- `value = [384, 44]`:  Current class balance. There are 384 observations of class 0 and 44 observations of class 1.
- `samples = 428`:  Number of samples assessed at this node.
- `gini = 0.184`: Gini impurity, a measure of "impurity". The higher the value, the bigger the mix of classes. A 50/50 split of two classes would result in an index of 0.5.
- `acutePhysiologyScore <=78.5`: Decision rule learned by the node. In this case, patients with a score of <= 78.5 are moved into the left node and >78.5 to the right. 

The gini impurity is is actually used by the algorithm to determine a split. The model evaluates every feature (in our case, age and score) at every possible split (46, 47, 48..) to find the point with the lowest gini impurity in two resulting nodes. 

The approach is referred to as "greedy" because we are choosing the optimal split given our current state. Let's take a closer look at our decision boundary.

In [0]:
# look at the regions in a 2d plot
# based on scikit-learn tutorial plot_iris.html
plt.figure(figsize=[10,8])
dtn.plot_model_pred_2d(mdl, X_train, y_train)

In this plot we can see the decision boundary on the y-axis, separating the predicted classes. The true classes are indicated at each point. Where the background and point colours are mismatched, there has been misclassification. Of course we are using a very simple model. Let's see what happens when we increase the depth.

In [0]:
mdl = tree.DecisionTreeClassifier(max_depth=5)
mdl = mdl.fit(X_train,y_train)

In [0]:
plt.figure(figsize=[10,8])
dtn.plot_model_pred_2d(mdl, X_train, y_train)

Now our tree is more complicated! We can see a few vertical boundaries as well as the horizontal one from before. Some of these we may like, but some appear unnatural. Let's look at the tree itself.

In [0]:
graph = dtn.create_graph(mdl,feature_names=features)
Image(graph.create_png())

Looking at the tree, we can see that there are some very specific rules. Consider our patient aged 65 years with an acute physiology score of 87. From the top of the tree, we would work our way down:

- acutePhysiologyScore <= 78.5? No.
- acutePhysiologyScore <= 106.5? Yes.
- age <= 75.5? Yes
- age <= 66. Yes.
- age <= 62.5? No. 

This leads us to our single node with a gini impurity of 0. Having an entire rule based upon this one observation seems silly, but it is perfectly logical as at the moment. The only objective the algorithm cares about is minimizing the gini impurity. 

We are at risk of overfitting our data! This is where "pruning" comes in.

In [0]:
# let's prune the model and look again
dtn.prune(mdl, min_samples_leaf = 10)
graph = dtn.create_graph(mdl,feature_names=features)
Image(graph.create_png())  

Above, we can see that our second tree is (1) smaller in depth, and (2) never splits a node with <= 10 samples. We can look at the decision surface for this tree:

In [0]:
plt.figure(figsize=[10,8])
dtn.plot_model_pred_2d(mdl, X_train, y_train)

Our pruned decision tree has a much more intuitive boundary, but does make some errors. We have reduced our performance in an effort to simplify the tree. This is the classic machine learning problem of trading off complexity with error.

Note that, in order to do this, we "invented" the minimum samples per leaf node of 10. Why 10? Why not 5? Why not 20? The answer is: it depends on the dataset. Heuristically choosing these parameters can be time consuming, and we will see later on how gradient boosting elegantly handles this task.

## Decision trees have high "variance"

Before we move on to boosting, it will be useful to demonstrate how decision trees have high "variance". In this context, variance refers to a property of some models to have a wide range of performance given random samples of data. Let's take a look at randomly slicing the data we have too see what that means.

In [0]:
np.random.seed(123)

fig = plt.figure(figsize=[12,3])

for i in range(3):
    ax = fig.add_subplot(1,3,i+1)

    # generate indices in a random order
    idx = np.random.permutation(X_train.shape[0])
    
    # only use the first 50
    idx = idx[:50]
    X_temp = X_train.iloc[idx]
    y_temp = y_train.values[idx]
    
    # initialize the model
    mdl = tree.DecisionTreeClassifier(max_depth=5)
    
    # train the model using the dataset
    mdl = mdl.fit(X_temp, y_temp)
    dtn.plot_model_pred_2d(mdl, X_temp, y_temp, cbar=False)

Above we can see that we are using random subsets of data, and as a result, our decision boundary can change quite a bit. As you could guess, we actually don't want a model that randomly works well and randomly works poorly, so you may wonder why this is useful. 

The trick is that by combining many of instances of "high variance" classifiers (decision trees), we can end up with a single classifier with low variance. There is an old joke: two farmers and a statistician go hunting. They see a deer: the first farmer shoots, and misses to the left. The next farmer shoots, and misses to the right. The statistician yells "We got it!!".

While it doesn't quite hold in real life, it turns out that this principle does hold for decision trees. Combining them in the right way ends up building powerful models.

## Boosting

The premise of boosting is the combination of many weak learners to form a single "strong" learner. In a nutshell, boosting involves building a models iteratively, and at each step we focus on the data we performed poorly on. In our context, we'll use decision trees, so the first step would be to build a tree using the data. Next, we'd look at the data that we misclassified, and re-weight the data so that we really wanted to classify those observations correctly, at a cost of maybe getting some of the other data wrong this time. Let's see how this works in practice.

In [0]:
# build the model
clf = tree.DecisionTreeClassifier(max_depth=1)
mdl = ensemble.AdaBoostClassifier(base_estimator=clf,n_estimators=6)
mdl = mdl.fit(X_train,y_train)

# plot each individual decision tree
fig = plt.figure(figsize=[10,5])
for i, estimator in enumerate(mdl.estimators_):
    ax = fig.add_subplot(2,3,i+1)
    dtn.plot_model_pred_2d(estimator, X_train, y_train, cbar=False)

Looking at the above, we can see that the first iteration builds the exact same simple decision tree as we had seen earlier. This makes sense - it's using the entire dataset with no special weighting. 

In the next iteration we can see the model shift - it misclassified five observations in class 1, and now these are the most important observations. Consequently, it picks the boundary that, while prioritizing correctly classifies these observations, still tries to best classify the rest of the data too. Now we have correctly classified all but one observation, the one on the far left middle of the graph. In iteration 3, the algorithm solely focuses on correctly classifying this one observation.

One important point is that each tree is weighted by it's global error. In the figure above, it's obvious that we wouldn't want to weight Tree 3 equally to Tree 1, when Tree 1 is doing so much better overall. It turns out that weighting each tree by the inverse of its error is a pretty good way to do this.

Let's look at final model's decision surface.


In [0]:
# plot the final prediction
plt.figure(figsize=[9,5])
dtn.plot_model_pred_2d(mdl, X_train, y_train)
plt.show()

And that's AdaBoost! There are a few tricks we have glossed over here - but you understand the general principle. Now we'll move on to a different approach. With boosting, we iteratively changed the dataset to have new trees focus on the "difficult" observations. The next approach we discuss is similar as it also involves using changed versions of our dataset to build new trees.